# Polygenic Risk Score Analysis on Height GWAS data

Here we show an example of our [LDpred2 pipeline](https://github.com/cumc/bioworkflows/blob/master/ldpred/ldpred.ipynb) for height PRS on [an example public data-set](https://drive.google.com/file/d/1x_G0Gxk9jFMY-PMqwtg6-vdEyUPp5p5u/view) from github user [@choishingwan](https://github.com/choishingwan).

Analysis in this tutorial is performed by running our pipeline sequentially on a single computer (a desktop, laptop, or a single interactive node on a cluster). The pipeline iteself, implemented in the SoS workflow language, can be configured to run in parallel on a high performance computing cluster environment. Please read the [SoS documentation](https://vatlab.github.io/sos-docs/doc/user_guide/host_setup.html) on how to configure the software and workflow to efficiently perform the analysis for real-world data.

## Data used

### Reference panel

Obtained via `download_1000G()` in `bigsnpr`. 

We extracted unrelated European individuals (~500  samples) and ~1.7M SNPs in common with either HapMap3 or the UK Biobank. Classification of European population can be found at [IGSR](https://www.internationalgenome.org/category/population/). European individuals ID are from [IGSR data portal](https://www.internationalgenome.org/data-portal/sample).

### GWAS summary statistics data

`Height.QC.gz`, from public data-set provided by github user [@choishingwan](https://github.com/choishingwan), of height GWAS in European samples.

### Target test data

`EUR.height`, `EUR.cov`, and `EUR.eigenvec` contain phenotypes, covariates and genotype principle components of samples. `EUR.QC.*` contain the corresponding genotypes, with ~400,000 variants. We have thinned the genotype data to ~150,000 variants for speeding up the examples used in this tutorial. The resulting genotypes are in `EUR_prune.*`.

## PRS Models applied

Auto model runs the algorithm for 30 different $p$ (the proportion of causal variants) values range from 10e-4 to 0.9, and heritability $h^2$ from LD score regression as initial value.

Grid model tries a grid of parameters $p$, ranges from 0 to 1 and three $h^2$ which are 0.7/1/1.4 times of initial $h^2$ estimated by LD score regression.

## Data preparation

Please download the [tutorial data](http://statgen.us/files/2021/01/ldpred2.tar.gz) and [pipeline script](https://github.com/cumc/bioworkflows/blob/master/ldpred/ldpred.ipynb) to your computer. We have pre-downloaded and extracted the European genotypes from 1000 Genomes, as well as genetic map data. We have preprocessed the GWAS data as follows, to fit in our pipeline.

**Please do not run any code from this section. The data bundle above contains all the preprocessed files for you to start the PRS analysis (next section). Code below are documented here for book-keeping and reproducibility.**

### Summary statistics formatting

In [ ]:
sumstats <- bigreadr::fread2("GWAS_data/Height.QC.gz") 
# LDpred2 require the header to follow the exact naming
names(sumstats) <-
    c("chr",
    "pos",
    "rsid",
    "a1",
    "a0",
    "n_eff",
    "beta_se",
    "p",
    "OR",
    "INFO",
    "MAF")
# Transform the OR into log(OR)
sumstats$beta <- log(sumstats$OR)

In [ ]:
saveRDS(sumstats, "GWAS_data/Height.QC.rds")

### Phenotype and covariates formatting

In [ ]:
options(stringsAsFactors=F)
fam = read.table("GWAS_data/EUR_prune.fam", header=F)
colnames(fam) = c("FID", "IID", "PID", "MID", "S", "D")
pheno = read.table("GWAS_data/EUR.height", header=T)
covar = read.table("GWAS_data/EUR.cov", header=T)
pcs = read.table("GWAS_data/EUR.eigenvec", header=F)
colnames(pcs) = c("FID", "IID", "PC1", "PC2", "PC3", "PC4", "PC5", "PC6")

In [ ]:
require(dplyr)
pheno_out = left_join(fam, pheno, by = c("FID", "IID"))
pheno_out = left_join(pheno_out, covar, by = c("FID", "IID"))
pheno_out = left_join(pheno_out, pcs, by = c("FID", "IID"))

In [ ]:
write.table(pheno_out[, "Height"], "GWAS_data/EUR.height.matched.txt", col.names="Height",row.names=F)
write.table(pheno_out[, c("Sex",'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6')], "GWAS_data/EUR.cov.matched.txt", col.names=T,row.names=F)

The directory should have the following:

In [ ]:
tree

## Analysis of height GWAS data

We set the work directory to `height_results` folder (to be created by the workflow). This will also be used as part of the filenames in the outputs to identify this analysis.

In [ ]:
work_dir="height_results"

### Step 1: QC on reference panel

Here we assume the GWAS genotype data `EUR.*` has already been QC-ed. We perform here QC for reference panel,

In [ ]:
sos run ldpred.ipynb snp_qc \
    --cwd $work_dir \
    --genoFiles 1000G.EUR/1000G.EUR.bed

The output of a command in our pipeline is highlighted in the green text above, in this case, ` /home/jovyan/work/height_results/1000G.EUR.height_results.bed`. These are typically intermediate files generated and kept by the pipeline program. In this tutorial we will show contents from these outputs only when they are relevant to the final results, although in practice users are always encouraged to check these immediate files to ensure of no obvious sign of problems in each analysis step.

### Step 2: Intersect SNPs among summary stats, reference panel and target data

SNPs shared between summary statistics, reference panels and target genotype data (for which PRS will be computed) are extracted. Genetic distances will be computed for each variant using interpolated genetic map. **This step can take a bit of time to execute**.

In [ ]:
sos run ldpred.ipynb snp_intersect \
    --cwd $work_dir \
    --ss GWAS_data/Height.QC.rds \
    --genoFiles $work_dir/1000G.EUR.$work_dir.bed GWAS_data/EUR_prune.bed

In [ ]:
cat $work_dir/Height.QC.intersect.stdout

### Step 3: Harmonize alleles for shared SNPs

To handle major/minor allele, strand flips and consequently possible flips in sign for summary statistics.

In [ ]:
sos run ldpred.ipynb snp_match \
    --cwd $work_dir \
    --reference_geno $work_dir/1000G.EUR.$work_dir.snp_intersect.extracted.rds \
    --ss GWAS_data/Height.QC.rds

### Step 4: Summary statistics quality control (optional)

Please refer to documentation in the pipeline notebook [`ldpred.ipynb`](https://github.com/cumc/bioworkflows/blob/master/ldpred/ldpred.ipynb) for an explanation of summary statistics QC.

In [ ]:
sos run ldpred.ipynb sumstats_qc \
    --cwd $work_dir \
    --reference_geno $work_dir/1000G.EUR.$work_dir.snp_intersect.extracted.rds \
    --ss $work_dir/Height.QC.snp_matched.rds \
    --sdy 1

In [ ]:
%preview height_results/Height.QC.snp_matched.qc.png

From the results, we observe only a few outliers significantly deviating from the majority of the variants. We suspect that quality control recommanded in [LDpred2 manuscript](https://academic.oup.com/bioinformatics/article/36/22-23/5424/6039173) may be too stringent and should not be needed for this data-set. Notice that as the manuscript pointed out, the quality control procedure is not suitable for meta-analyzed GWAS summary statistics.

### Step 5: Calculate LD matrix and fit LDSC model

As an illustration, hereafter all analysis are performed on both summary statistics before QC in Step 4:

In [ ]:
sos run ldpred.ipynb ldsc \
    --cwd $work_dir \
    --ss $work_dir/Height.QC.snp_matched.rds \
    --reference-geno $work_dir/1000G.EUR.$work_dir.snp_intersect.extracted.rds

and after QC in Step 4:

In [ ]:
sos run ldpred.ipynb ldsc \
    --cwd $work_dir \
    --ss $work_dir/Height.QC.snp_matched.qc.rds \
    --reference-geno $work_dir/1000G.EUR.$work_dir.snp_intersect.extracted.rds

### Step 6: Estimate posterior effect sizes and PRS

For QC-ed data, we perform 3 PRS models implemented in `ldpred2`. We demonstrate the infinitesimal model here:

In [ ]:
sos run ldpred.ipynb inf_prs \
    --cwd $work_dir \
    --ss $work_dir/Height.QC.snp_matched.qc.rds \
    --target-geno $work_dir/EUR_prune.snp_intersect.extracted.rds \
    --ldsc $work_dir/Height.QC.snp_matched.qc.ld.rds

Here we examine the PRS generated from this command,

In [ ]:
cat $work_dir/Height.QC.snp_matched.qc.inf_prs.stdout

In [ ]:
dat = readRDS('height_results/Height.QC.snp_matched.qc.inf_prs.rds')
names(dat)

In [ ]:
hist(dat$prs, breaks=20, main="PRS distribution", xlab="PRS")

Commands below will execute the "auto" model and "grid" model implemented in `LDpred2`. They are typically more powerful but also computationally intensive. We document them here without executing:

```
sos run ldpred.ipynb auto_prs \
    --cwd $work_dir \
    --ss $work_dir/Height.QC.snp_matched.qc.rds \
    --target-geno $work_dir/EUR_prune.snp_intersect.extracted.rds \
    --ldsc $work_dir/Height.QC.snp_matched.qc.ld.rds
```

```
sos run ldpred.ipynb grid_prs \
    --cwd $work_dir \
    --ss $work_dir/Height.QC.snp_matched.qc.rds \
    --target-geno $work_dir/EUR_prune.snp_intersect.extracted.rds \
    --ldsc $work_dir/Height.QC.snp_matched.qc.ld.rds \
    --phenoFile GWAS_data/EUR.height.matched.txt \
    --covFile GWAS_data/EUR.cov.matched.txt \
    --response continuous
```

We then run the infinitesimal model for original data:

In [ ]:
sos run ldpred.ipynb inf_prs \
    --cwd $work_dir \
    --ss $work_dir/Height.QC.snp_matched.rds \
    --target-geno $work_dir/EUR_prune.snp_intersect.extracted.rds \
    --ldsc $work_dir/Height.QC.snp_matched.ld.rds

In [ ]:
cat $work_dir/Height.QC.snp_matched.inf_prs.stdout

In [ ]:
dat = readRDS('height_results/Height.QC.snp_matched.inf_prs.rds')
hist(dat$prs, breaks=20, main="PRS distribution (without summary statistics QC)", xlab="PRS")

Again, here are commands for "auto" and "grid" models:

```
sos run ldpred.ipynb auto_prs \
    --cwd $work_dir \
    --ss $work_dir/Height.QC.snp_matched.rds \
    --target-geno $work_dir/EUR_prune.snp_intersect.extracted.rds \
    --ldsc $work_dir/Height.QC.snp_matched.ld.rds
```

```
sos run ldpred.ipynb grid_prs \
    --cwd $work_dir \
    --ss $work_dir/Height.QC.snp_matched.rds \
    --target-geno $work_dir/EUR_prune.snp_intersect.extracted.rds \
    --ldsc $work_dir/Height.QC.snp_matched.ld.rds \
    --phenoFile GWAS_data/EUR.height.matched.txt \
    --covFile GWAS_data/EUR.cov.matched.txt \
    --response continuous
```

**For `grid` model, in practice we should use another subset of individuals to train the model, independent from the subset to make PRS predictions. Here we use the same target genotype data only to illustrate the workflow.**

### Step 7: predict phenotypes

Baseline model: Trait ~ Sex + PCs

In [ ]:
sos run ldpred.ipynb pred_eval \
    --cwd $work_dir \
    --phenoFile GWAS_data/EUR.height.matched.txt \
    --covFile GWAS_data/EUR.cov.matched.txt \
    --response continuous

In [ ]:
res = readRDS("height_results/EUR.height.matched.baseline.rds")
summary(res$fitted)
res$summary

Infinitesimal model: Trait ~ Sex + PCs + PRS

In [ ]:
sos run ldpred.ipynb pred_eval \
    --cwd $work_dir \
    --prs $work_dir/Height.QC.snp_matched.qc.inf_prs.rds \
    --phenoFile GWAS_data/EUR.height.matched.txt \
    --covFile GWAS_data/EUR.cov.matched.txt \
    --response continuous

sos run ldpred.ipynb pred_eval \
    --cwd $work_dir \
    --prs $work_dir/Height.QC.snp_matched.inf_prs.rds \
    --phenoFile GWAS_data/EUR.height.matched.txt \
    --covFile GWAS_data/EUR.cov.matched.txt \
    --response continuous

In [ ]:
res = readRDS("height_results/EUR.height.matched.Height.QC.snp_matched.qc.inf_prs.rds")
summary(res$fitted)
res$summary

In [ ]:
res = readRDS("height_results/EUR.height.matched.Height.QC.snp_matched.inf_prs.rds")
summary(res$fitted)
res$summary

Here are commands for running prediction with "auto" and "grid" models:

```
sos run ldpred.ipynb pred_eval \
    --cwd $work_dir \
    --prs $work_dir/Height.QC.snp_matched.qc.auto_prs.rds \
    --phenoFile GWAS_data/EUR.height.matched.txt \
    --covFile GWAS_data/EUR.cov.matched.txt \
    --response continuous

sos run ldpred.ipynb pred_eval \
    --cwd $work_dir \
    --prs $work_dir/Height.QC.snp_matched.auto_prs.rds \
    --phenoFile GWAS_data/EUR.height.matched.txt \
    --covFile GWAS_data/EUR.cov.matched.txt \
    --response continuous

sos run ldpred.ipynb pred_eval \
    --cwd $work_dir \
    --prs $work_dir/Height.QC.snp_matched.qc.grid_prs.rds \
    --phenoFile GWAS_data/EUR.height.matched.txt \
    --covFile GWAS_data/EUR.cov.matched.txt \
    --response continuous

sos run ldpred.ipynb pred_eval \
    --cwd $work_dir \
    --prs $work_dir/Height.QC.snp_matched.grid_prs.rds \
    --phenoFile GWAS_data/EUR.height.matched.txt \
    --covFile GWAS_data/EUR.cov.matched.txt \
    --response continuous
```

## Results summary

Following table shows adjusted R squared of height prediction models. "QC" refers to quality control in Step 4.

Compared to baseline model, higher R squared with PRS included implies PRS explains part of the variation of height in the target data-set. Higher R squared is observed without performing quality control for the summary statistics, as we already expected and discussed after seeing the results in Step 4. 

| QC? |   # of SNPs  |   Baseline R2 |   Inf R2  |   Grid R2 |   Auto R2 |
|:---:|:------------:|:-------:|:-------:|:-------:|:-------:|
| Yes | 104,564 | 0.2296 | 0.3581 | -  | - |
|  No | 109,963 | 0.2296 | 0.3619 |    -  |    -   |

## Homework

Please complete the runs with `grid` and `auto` model and complete the table above. **Computation with these models are intensive and long-running; it is suggested that the analysis be performed in a powerful computing environment**.